In [1]:
import SPARQLWrapper

In [67]:
city_list = ["Paris", "Rome", "Berlin", "Florence", "London", "Liverpool", "Manchester"]

In [52]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_ambiguous_city(city):
    search_city = city + "_(disambiguation)"
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?dis

        WHERE { <http://dbpedia.org/resource/""" + search_city+"""> dbo:wikiPageDisambiguates ?dis .
               ?dis a dbo:City}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()


    collect = []
    for result in results["results"]["bindings"]:
        
        saving = result["dis"]["value"]
        if saving == "http://dbpedia.org/resource/" + city:
            continue
        
        collect.append(saving)
        
        
    return collect

In [66]:
get_ambiguous_city("Manchester")

['http://dbpedia.org/resource/Manchester,_Georgia',
 'http://dbpedia.org/resource/Manchester,_Iowa',
 'http://dbpedia.org/resource/Manchester,_Kansas',
 'http://dbpedia.org/resource/Manchester,_Kentucky',
 'http://dbpedia.org/resource/Manchester,_Minnesota',
 'http://dbpedia.org/resource/Manchester,_Missouri',
 'http://dbpedia.org/resource/Manchester,_New_Hampshire',
 'http://dbpedia.org/resource/Manchester,_Tennessee']

In [46]:
def get_born_ambiguous_city(city):
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?dis ?date ?country
        WHERE { ?dis dbo:birthPlace <http://dbpedia.org/resource/""" + city+"""> . 
                <http://dbpedia.org/resource/""" + city+"""> dbo:country ?country .
                ?dis dbo:birthDate ?date}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()


    collect = []
    for result in results["results"]["bindings"]:
        
        saving = result["dis"]["value"]
        if saving == "http://dbpedia.org/resource/" + city:
            continue
        collect.append((saving, result["date"]["value"], city, result["country"]["value"]))
        
    collect = list(filter(lambda x : "_(" in x[0], collect))
    return collect

In [51]:
for a,b,c,d in get_born_ambiguous_city("Berlin,_Wisconsin"):
    print(a.split("/")[-1],b,c,d)

Bill_Butler_(American_football) 1937-07-10 Berlin,_Wisconsin http://dbpedia.org/resource/United_States
Bill_Butler_(American_football) 1937-7-10 Berlin,_Wisconsin http://dbpedia.org/resource/United_States
